In [1]:
import numpy as np
from helpers import *
from implementations import *

In [2]:
# loading train data (takes 10 secs, avoid spamming cell)

yb, input_data, ids = load_csv_data("train.csv")
dimensions = np.shape(input_data)
N = dimensions[0]
P = dimensions[1]
yb = np.reshape(yb,[N,1])


In [58]:
# standardize data without the incomplete points, calculations done on _rem arrays (with -999 lines removed)
idx_incomplete_points = np.nonzero(input_data[:,4]==-999)

tx_rem = np.delete(input_data,idx_incomplete_points,0)
y_rem = np.delete(yb,idx_incomplete_points)

means = np.mean(tx_rem, axis=0)
means = means * np.ones(np.shape(tx_rem))
tx_rem_std = tx_rem - means

std_dev = np.std(tx_rem, axis=0)
std_dev = std_dev * np.ones(np.shape(tx_rem))
tx_rem_std = tx_rem_std / std_dev       # np.mean(tx_rem_std) should be ~0, np.std(tx_rem_std) should be 1


In [59]:
np.shape(tx_rem_std)


(72543, 30)

In [60]:
tx_train = tx_rem_std[0:100,:]  # 100x30 data for faster testing of regression
y_train = yb[0:100]


In [61]:
initial_w = 3*np.ones([30,1])
max_iters = 10
gamma = 0.9
w_opt,loss = logistic_regression(y_train,tx_train,initial_w,max_iters,gamma)

In [62]:
np.shape(initial_w)

(30, 1)

In [63]:
loss

3.480514748930746

In [64]:
tx_train

array([[ 0.30601378,  0.33589804,  0.43671424, ...,  0.61614788,
        -1.36131161, -0.63958218],
       [ 0.12880888, -0.71689763, -0.46648342, ...,  0.11608109,
         1.71034105,  0.10556264],
       [ 0.34341424, -0.29384535,  0.66918057, ...,  0.07030726,
        -1.52202162, -0.02255546],
       ...,
       [ 0.49980958, -0.97072347,  1.13770939, ..., -1.05484302,
         0.90018647,  0.08191522],
       [ 0.14045016, -0.73772975, -0.23219565, ..., -1.29257163,
        -0.46639901,  1.29077821],
       [ 0.28828092,  0.73389099,  0.51688068, ..., -1.11439822,
        -0.88028233, -0.73337247]])